# Residency Project Deliverable: 1
# Rutu Shah
# Nischal Joshi
# Murali Krishna
# Santhosh Ramachandran 

In [12]:
import pandas as pd


In [14]:
df = pd.read_csv('The Titanic dataset.csv')

In [13]:


#explore the dataset
print("Printing head()\n", df.head())
print("Printing describe()\n", df.describe())
print("Printing info()\n", df.info())


Printing head()
     1       2         3                            4       5    6       7  \
0  sn  pclass  survived                          NaN  gender  age  family   
1   1       3         0                  Mr. Anthony    male   42       0   
2   1       3         0                  Mr. Anthony    male   42       0   
3   2       3         0        Master. Eugene Joseph    male    ?       2   
4   3       2         0  Abbott, Mr. Rossmore Edward     NaN  NaN       2   

       8         9         10  
0   fare  embarked       date  
1   7.55       NaN  01-Jan-90  
2   7.55       NaN  01-Jan-90  
3  20.25         S  02-Jan-90  
4     **         S  03-Jan-90  
Printing describe()
            1     2     3            4     5     6     7     8     9         10
count   1302  1302  1302         1301  1301  1045  1300  1300  1296       1302
unique  1301     4     3         1297     3    98    10   283     4       1301
top        1     3     0  Mr. Anthony  male    24     0    13     S  0

In [8]:
# Check for missing values in each column
print(df.isnull().sum())


1       0
2       0
3       0
4       1
5       1
6     257
7       2
8       2
9       6
10      0
dtype: int64
